In [1]:
import json
import os
import sklearn
import numpy
import xgboost as xgb
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from collections import namedtuple
import sys
from math import log,tan,pi,radians
import pandas as pd
from sodapy import Socrata
from config import CHI_API_KEY
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv("hoods.csv")

In [3]:
df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [5]:
df['hood_label'].value_counts()
df = df[df.hood_label != 0]

In [6]:
df = df[df['hood_label'].notna()]
df = df[df['weather_condition'].notna()]
df = df[df['lighting_condition'].notna()]
df = df[df['crash_hour'].notna()]
df = df[df['crash_day_of_week'].notna()]
df = df[df['posted_speed_limit'].notna()]
df['hit_and_run_i'] = df['hit_and_run_i'].fillna('N')

In [7]:
df['hit_and_run_i'].value_counts()

N    321361
Y    124495
Name: hit_and_run_i, dtype: int64

In [8]:
df['crash_day_of_week'] = df['crash_day_of_week'].replace([1,2,3,4,5,6,7],['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'])
df['crash_day_of_week'].value_counts()

Friday       72489
Saturday     65749
Tuesday      64135
Thursday     63948
Wednesday    63451
Monday       62026
Sunday       54058
Name: crash_day_of_week, dtype: int64

In [9]:
df['crash_hour'] = df['crash_hour'].replace([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24],['Midnight', '1am', '2am', '3am', '4am', '5am', '6am', '7am', '8am', '9am', '10am', '11am', 'Noon', '1pm', '2pm', '3pm', '4pm',
                                                                                                                    '5pm', '6pm', '7pm', '8pm', '9pm', '10pm', '11pm'])
df['crash_hour'].value_counts()

3pm         34157
2pm         34048
4pm         33607
1pm         30216
5pm         27730
Noon        27377
11am        26342
7am         24257
10am        22650
8am         20999
9am         20436
6pm         20204
6am         19669
7pm         15960
8pm         14288
9pm         13160
10pm        11028
5am         10081
0            8938
Midnight     7583
1am          6613
4am          6114
2am          5401
3am          4998
Name: crash_hour, dtype: int64

In [10]:
X = pd.get_dummies(df[['hood_label', "weather_condition", "lighting_condition", "crash_day_of_week", "crash_hour"]])

In [11]:
values = array(df["hood_label"])

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)


onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
hl_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [12]:
values = array(df["hit_and_run_i"])

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)


onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
damage_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [13]:
y = damage_onehot_encoded
print(X.shape)
print(y.shape)

(445856, 126)
(445856, 2)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [15]:
xgb_estimator = xgb.XGBClassifier(objective='binary:logistic')

In [16]:
multilabel_model = MultiOutputClassifier(xgb_estimator)

In [17]:
multilabel_model.fit(X_train, y_train)

MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None, gamma=None,
                                              gpu_id=None,
                                              importance_type='gain',
                                              interaction_constraints=None,
                                              learning_rate=None,
                                              max_delta_step=None,
                                              max_depth=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                         

In [18]:
print('Accuracy on test data: {:.1f}%'.format(accuracy_score(y_test, multilabel_model.predict(X_test))*100))

Accuracy on test data: 73.6%


In [19]:
# Save model to run in flask
from joblib import dump, load
dump(multilabel_model, 'hit_and_run.joblib') 

['hit_and_run.joblib']